### 수집된 포털 정보기반 딥크롤링
* 기존에 수집된 정보를 바탕으로 매체별 크롤링
* 다양한 서브 데이터의 정형화
* URL 기반의 다양한 크롤링 기법 사용

### 분석요소
* 제목 :
* 내용  :
* 작성일 :
* 매체: 
* 작성자:


### 기존 수집 데이터 활용


In [7]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [8]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [9]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [10]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [11]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select nidx,ncomp,nurl from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [23]:

def insSub(d):
    dbname='navernews.db'
    d[7] = makeKey(d[6])
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from subnews where skey="'+d[7]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[0])
        else:
            sql='insert into subnews (stitle,swriter,sday,sarticle,smidx,scompany,surl,skey) values(?,?,?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [41]:
def setStat(index=0,stat=0):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='update nnews set nstat ={1} where nidx={0}'.format(index,nstat)
        res=cursor.execute(sql)
        if stat==1:
            print('-',end='')
        else:
            print('=',end='')

In [42]:
setStat(67,1)

NameError: name 'nstat' is not defined

In [24]:
def getAsia(urls):
    url = urls[2]
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div .area_title')[0]
    wtitle=title_block.select('h3')[0].text.strip()
    writer=htmls.select('.article_view > .article .e_article')[0].text.strip()
    wday_=title_block.select('p.user_data')
    wday = str(wday_).split('</strong>')[-1].replace('</p>]','').replace('.','-').strip()
    article_=htmls.select('.article_view > .article ')[0]
    pattern='<!-- 본문// -->[\S\s]*'
    article__=re.sub(pattern,' ',str(article_))
#     pattern='<div id="bestnews_layer">.*'
#     article__=re.sub(pattern,' ',str(article__))
    warticle=bsp(article__,'html.parser').text.strip()
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]
    #return [wtitle,wday,urls[0],urls[1],urls[2]]
urls = (69,'아시아경제','https://view.asiae.co.kr/article/2021031715545233623')
#print(getAsia(urls))

In [25]:
def getDonga(urls):
    url = urls[2]
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div .article_title')[0]
    wtitle=title_block.select('h1.title')[0].text.strip()
    writer=title_block.select('span.report')[0].text.replace('동아닷컴 기자','').strip()
    wday=title_block.select('span.date01')[-1].text.strip()
    article_=htmls.select('div.article_txt')[0]
    pattern='<script[^>]*>((\n|\r|.)*?)<\/script>'
    article__=re.sub(pattern,' ',str(article_))
    pattern='<div id="bestnews_layer">.*'
    article__=re.sub(pattern,' ',str(article__))
    warticle=bsp(article__,'html.parser').text
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]


In [15]:
def getDona(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div .article_title')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div .news_wrap')
        subartis=tgt.select('ul .list_cluster > li .sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a .news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a .news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a .sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a .sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [12]:
!pip install selenium

### 비동기 페이지
* 크롬 드라이버 활용
* 다운로드 : https://sites.google.com/a/chromium.org/chromedriver/downloads
* 해당 크롬 버전에 맞는 드라이버 다운

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as bsp

driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
url = ('https://www.chosun.com/economy/market_trend/2021/03/17/7O5JTHZSJVAJBFMUWNJGK22XUA/')
driver.get(url)
driver.implicitly_wait(3)
html = driver.page_source
src = bsp(html,'html.parser')

In [4]:
url = ('https://www.chosun.com/economy/market_trend/2021/03/17/7O5JTHZSJVAJBFMUWNJGK22XUA/')
urls = ['327','조선일보',url]

In [5]:
def getChosun(urls):
    url = urls[2]
    driver.get(url)
    driver.implicitly_wait(3)
    html = driver.page_source
    src = bsp(html,'html.parser')
    wtitle = src.select('h1.article-header__headline')[0].text
    writer = src.select('a.article-byline__author')[0].text.replace('기자','').strip()
    wdate_ = src.select('div .article-dateline')[0].text.split("|")[-1]
    wday = wdate_.replace('수정','').replace('.','-').strip()+':00'
    warticle = src.select('section.article-body')[0].text.replace('\xad','')
    #urls : main_index, company, url
    return [wtitle,writer,wday,warticle,urls[0],urls[1],urls[2]]

In [6]:
res = getChosun(urls)
print(res)

['신세계·네이버 2500억 지분 교환… 쿠팡에 맞서는 온·오프라인 동맹', '변희원', '2021-03-17 03:00:00', '이커머스 시장을 차지하기 위한 온·오프라인 유통업체 간의 합종연횡이 본격화되고 있다.네이버와 신세계는 16일 2500억원 상당의 지분을 교환하는 방식의 사업 제휴를 발표했다. 네이버는 이날 신세계인터내셔널 지분 1000억원어치와 이마트 지분 1500억원을 취득한다고 공시했다. 지난 1월 정용진 신세계 부회장이 네이버 성남 사옥을 방문해 이해진 네이버 창업자 겸 글로벌투자책임자(GIO)를 만나 협력을 제안한 이후 두 달 만에 제휴가 구체화된 것이다.유통업계에서는 온라인 상거래 1위 업체 네이버와 오프라인 쇼핑 강자인 신세계가 예상보다 빨리 손을 잡은 것은 최근 쿠팡이 뉴욕 증시에 상장한데다 이베이코리아 인수전까지 벌어진 이커머스 판도에 발빠른 대응을 하기 위해서란 분석이 나온다. 신세계 관계자는 “현시점에 제휴를 하지 않으면 실기(失期)를 할 수 있단 생각에 (지분 교환이) 빠르게 진행됐다”고 했다.신세계의 온라인 쇼핑몰 ‘쓱닷컴’은 지난해 이커머스 시장 점유율이 2.4%에 불과하다. 시장 점유율 1위인 네이버쇼핑과 협력하면서 온라인 시장에서의 영향력을 키우겠다는 전략이다. 네이버 입장에서는 신세계 이마트가 강점을 갖고 있는 신선식품이나 도심 내 오프라인 매장을 기반으로 한 당일 배송을 본격적으로 추진할 수 있다. 한 업계 관계자는 “두 회사가 각각 가진 장점이 뚜렷한 만큼, 이번 제휴를 통해 서로의 약점을 보완하고 시너지 효과를 낼 수 있다고 판단한 것 같다”고 말했다. 네이버는 지난해 10월 CJ그룹과도 6000억원 규모의 지분을 교환한 바 있다.한편 이날 이베이코리아 예비 입찰도 마감됐다. 이마트, 롯데, 홈플러스 대주주인 MBK파트너스, SK텔레콤 등이 예비 입찰에 참여했다.', '327', '조선일보', 'https://www.chosun.com/economy/market_trend/2021/03/17/7O5JTHZSJVA

In [26]:
allurl=getData()
allurl[:5]
for urls in allurl[:3]:
    print('-'*50)
    wdata=''
    #print(nidx,ncomp,turl)
    if urls[1]=="동아일보":
        wdata=getDonga(urls)
    elif urls[1]=="아시아경제":
        wdata=getAsia(urls)
    elif urls[1]=="조선일보":
        wdata=getChosun(urls)
    print(wdata)

--------------------------------------------------

--------------------------------------------------

--------------------------------------------------

